In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

sns.set(rc={'figure.figsize':(15,8)})

path_to_catalog=r'C:\Users\kgrzebien\Desktop\python_tasks\reklamacje\raport_felery\\'


In [2]:
df_slownik=pd.read_csv('../słowniki/slownik.csv',
                       encoding='Cp1250',
                      sep="|")
df_slownik=df_slownik.loc[df_slownik['IS_DELETED']=='N'][['KOD_TEMA', 'NAZWA','RODZAJ','ZNACZNIK', 'TRZY', 'NAZWA_IMP', 'CENA_WAL', 'SYMBOL_IMP']]
df_slownik_znacznik=pd.read_csv('../słowniki/znacznik.csv',
                       encoding='Cp1250',
                      sep="|")
df_slownik_znacznik=df_slownik_znacznik.loc[df_slownik_znacznik['IS_DELETED']=='N'][['NAZWA','ZNACZNIK']]
df_slownik_znacznik=df_slownik_znacznik.rename(columns={'NAZWA':'NZNACZ'})

In [3]:
def func_read_data(path):
    df=pd.read_csv(path+'.csv',
              encoding='Cp1250',
              sep="|")
    return df
    
def func_str_to_float(s):
    s=s.astype(str)
    s=s.apply(lambda x: x.replace(',','.'))
    s=s.astype(float)
    return s

def func_add_time(df):
    
    df['Z_DNIA']=pd.to_datetime(df['Z_DNIA'], format='%y/%m/%d')
    
    df['year']=df['Z_DNIA'].apply(lambda x: x.year)
    df['month']=df['Z_DNIA'].apply(lambda x: x.month)
    
    #df=df.loc[df['Z_DNIA']<=np.datetime64('2021-05-13')]
    
    # df['day_of_year']=df['Z_DNIA'].apply(lambda x: x.timetuple().tm_yday)
    # df['week']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[1])
    # df['week_day']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[2])
    # df['year_month']=df['year'].astype(str)+'-'+df['month'].astype(str)+'-'+'01'
    # df['year_month']=df['year_month'].apply(lambda x: x[:5]+'0'+x[5:] if len(x)==9 else x)
    # df['year_month']=pd.to_datetime(df['year_month'], format='%Y-%m-%d')
    # df['year_week']=df['year'].astype(str)+'--'+df['week'].astype(str)
    # df['year_week']=df['year_week'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)==7 else x)
    # df['year_day']=df['year'].astype(str)+'--'+df['day_of_year'].astype(str)
    # df['year_day']=df['year_day'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)<9 else x)
    return df

def func_correct_name(df, name_col):
    list_of_name=['ZESTAW UPOMIN.']
    list_of_correct_name=['ZESTAWUPOMIN.']
    for zla_nazwa, dobra_nazwa in zip(list_of_name, list_of_correct_name):
        df[name_col]=df[name_col].replace(zla_nazwa, dobra_nazwa, regex=False)
    return df

In [21]:

    
def func_filtr_feler2(df, name_col, name_rodzaj):
    df['czy_z_f']=df[name_col].apply(lambda x: 'tak' if x[:2]=='F/' else 'nie')
    df['name_tmp']=df[name_col].apply(lambda x: x[2:] if x[:2]=='F/' else x)
    df_tmp_f=df.loc[df['czy_z_f']=='tak'].loc[df['ILOSC1']==0].loc[df['ILOSC2']>0]
    df=df.loc[df['czy_z_f']=='nie']
    display(len(df_tmp_f))
    # display(df_tmp_f.loc[df_tmp_f['name_tmp']=='ZESTAWUPOMIN.'])
    # display(df.loc[df['name_tmp']=='ZESTAWUPOMIN.'])
    df['czy_jest_taki_felere']='nie'
    df_tmp_f['czy_dodany']='nie'
    df_tmp_f['NOWA_NAZWA']=''
    df_tmp_f['NOWA_RODZAJ']=''
    df_tmp_f['NOWA_KOD_TEMA']=''
    list_od_error_faktury=[]
    for name, rodzaj, ind in tqdm(zip(df_tmp_f['name_tmp'], df_tmp_f[name_rodzaj], df_tmp_f.index)):
        try:
            if df.loc[df[name_col].str.contains(name)].loc[df[name_rodzaj]==rodzaj].empty:
                continue
            else:
                df_tmp_f.loc[ind, 'NOWA_NAZWA']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df[name_rodzaj]==rodzaj].index][name_col].values[0]
                df_tmp_f.loc[ind, 'NOWA_RODZAJ']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df[name_rodzaj]==rodzaj].index][name_rodzaj].values[0]
                df_tmp_f.loc[ind, 'NOWA_KOD_TEMA']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df[name_rodzaj]==rodzaj].index]['KOD_TEMA'].values[0]
                # df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index, 'czy_jest_taki_felere']='tak'
                # df_tmp_f.loc[ind, 'czy_dodany']='tak'
        except:
            list_od_error_faktury=np.append(list_od_error_faktury, df_tmp_f.loc[ind, 'NOWA_NAZWA'])
     
    # df=df.loc[df['czy_jest_taki_felere']=='tak']
    # display(len(df))
    # df=df.append(df_tmp_f.loc[df_tmp_f['czy_dodany']=='nie'], ignore_index=True)
    return df_tmp_f.reset_index().drop('index', axis=1), list_od_error_faktury

In [5]:
def func_raport(df, df_slownik, df_slownik_znacznik):
    df=df.merge(df_slownik_znacznik, how='left', on='ZNACZNIK')
    df=df.rename(columns={'NOWA_NAZWA':'NAZWA',
                          'NOWA_RODZAJ':'RODZAJ',
                          'ILOSC2':'ILOSC'})
    for i in tqdm(df.index):
        if df.loc[i,'NAZWA']=='':
            df.loc[i,'NAZWA']=df.loc[i,'NAZWA1']
            df.loc[i,'RODZAJ']=df.loc[i,'RODZAJ1']
    return df[['ZNACZNIK','NZNACZ','Z_DNIA','NAZWA','RODZAJ','SYMBOL_IMP','NAZWA_IMP', 'TRZY', 'ILOSC', 'CENA_WAL']]

In [4]:
### to jest zle bo kolumny z nazwami maja tylko 16 znakow

# def func_filtr_feler(df, name_col):
#     df['name_tmp']=df[name_col].apply(lambda x: x[2:] if x[:2]=='F/' else x)
#     df['czy_z_f']=df[name_col].apply(lambda x: 'tak' if x[:2]=='F/' else 'nie')
    
#     df_tmp=df.groupby(['FAKTURA','name_tmp']).agg({name_col:'nunique'}).reset_index()
#     df_tmp=df_tmp.loc[df_tmp[name_col]==2].drop(name_col, axis=1)
#     df=df.merge(df_tmp, how='inner', on=['FAKTURA','name_tmp'])
#     df=df.loc[df['czy_z_f']=='tak'].reset_index().drop('index', axis=1)
#     return df

In [29]:
df_test=pd.read_excel('raport_felery/FELERY2020_05_2021.xlsx')
#df_test=df_test.loc[df_test['ZNACZNIK']==19]
df_test['name_with_f']=df_test['NAZWA'].apply(lambda x: x if x[:2]=='F/' else 'F/'+x)
df_test['name_without_f']=df_test['NAZWA'].apply(lambda x: x[2:] if x[:2]=='F/' else x)


In [6]:
df=func_read_data('raport_felery/f56szac_21_06_22_06')
df['CENA_Z1']=func_str_to_float(df['CENA_Z1'])
df['CENA_Z2']=func_str_to_float(df['CENA_Z2'])
df['ILOSC1']=func_str_to_float(df['ILOSC1'])
df['ILOSC2']=func_str_to_float(df['ILOSC2'])
df['CENA_WAL']=func_str_to_float(df['CENA_WAL'])
df['CENA']=func_str_to_float(df['CENA'])
df=func_add_time(df)
df=func_correct_name(df, 'NAZWA1')

df_belki=func_read_data('raport_felery/f60szac_21_06_22_06')
df_belki['CENA_Z1']=func_str_to_float(df_belki['CENA_Z1'])
df_belki['CENA_Z2']=func_str_to_float(df_belki['CENA_Z2'])
df_belki['ILOSC1']=func_str_to_float(df_belki['ILOSC1'])
df_belki['ILOSC2']=func_str_to_float(df_belki['ILOSC2'])
df_belki['CENA_WAL']=func_str_to_float(df_belki['CENA_WAL'])
df_belki['CENA']=func_str_to_float(df_belki['CENA'])
df_belki=func_add_time(df_belki)
df_belki=func_correct_name(df_belki, 'NAZWA1')
#df=df.loc[df['ZNACZNIK']==19]

In [ ]:
#### brac tylko nazwa2 rodzaj2 i ilosc1==0 ilosc2>0

In [40]:
_1, list_od_error_faktury_1=func_filtr_feler2(df, 'NAZWA1','RODZAJ1')
df_raport1=func_raport(_1, df_slownik, df_slownik_znacznik)

8813

8813it [07:49, 18.78it/s]
100%|███████████████████████████████████████████████████████████████████████████| 8813/8813 [00:00<00:00, 28225.98it/s]


In [41]:
_2, list_od_error_faktury_2=func_filtr_feler2(df, 'NAZWA2','RODZAJ2')
df_raport2=func_raport(_2, df_slownik, df_slownik_znacznik)

8750

8750it [09:06, 16.01it/s]
100%|███████████████████████████████████████████████████████████████████████████| 8750/8750 [00:00<00:00, 27748.14it/s]


In [34]:
_belki1, list_od_error_faktury_belki1=func_filtr_feler2(df_belki, 'NAZWA1', 'RODZAJ1')
df_raport_belki1=func_raport(_belki1, df_slownik, df_slownik_znacznik)

1636

1636it [03:41,  7.38it/s]
100%|███████████████████████████████████████████████████████████████████████████| 1636/1636 [00:00<00:00, 17161.53it/s]


In [22]:
_belki2, list_od_error_faktury_belki2=func_filtr_feler2(df_belki, 'NAZWA2','RODZAJ2')

df_raport_belki2=func_raport(_belki2, df_slownik, df_slownik_znacznik)

10432

10432it [20:27,  8.50it/s]
100%|█████████████████████████████████████████████████████████████████████████| 10432/10432 [00:00<00:00, 16675.39it/s]


In [200]:
df_raport_belki2_new=\
df_raport_belki2.loc[df_raport_belki2['NAZWA_IMP']!=' '].append(
df_raport_belki2.loc[df_raport_belki2['NAZWA_IMP']==' '][['ZNACZNIK', 'NZNACZ', 'Z_DNIA', 'NAZWA', 'RODZAJ','ILOSC']].\
merge(df_slownik[~df_slownik.sort_values(by='KOD_TEMA', ascending=False)[['NAZWA', 'RODZAJ']].duplicated()][['NAZWA', 'RODZAJ', 'SYMBOL_IMP','NAZWA_IMP', 'TRZY', 'CENA_WAL']],
      how='left',
     on=['NAZWA', 'RODZAJ']),
    ignore_index=True)

In [199]:
df_raport2_new=\
df_raport2.loc[df_raport2['NAZWA_IMP']!=' '].append(
df_raport2.loc[df_raport2['NAZWA_IMP']==' '][['ZNACZNIK', 'NZNACZ', 'Z_DNIA', 'NAZWA', 'RODZAJ','ILOSC']].\
merge(df_slownik[~df_slownik.sort_values(by='KOD_TEMA', ascending=False)[['NAZWA', 'RODZAJ']].duplicated()][['NAZWA', 'RODZAJ', 'SYMBOL_IMP','NAZWA_IMP', 'TRZY', 'CENA_WAL']],
      how='left',
     on=['NAZWA', 'RODZAJ']),
    ignore_index=True)

In [204]:
df_raport2_new.append(df_raport_belki2_new, ignore_index=True).to_excel('raport_felery/df_raport2_felery_21_06_22_06.xlsx')

# testy

In [92]:
df_raport_belki.loc[df_raport_belki['NAZWA'].str.contains('193563/ZAS/J.S')].loc[df_raport_belki['Z_DNIA']==np.datetime64('2021-06-01')]

,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,SYMBOL_IMP,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
3,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,2.2,3.85
4,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,4.4,3.85
5,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,2.2,3.85
6,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,0.0,3.85


In [93]:
df_raport_belki.loc[df_raport_belki['ILOSC']>0]

,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,SYMBOL_IMP,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
0,1,BELKI,2021-06-01,F/193563/ZAS/CZA,280/HRT-38699,38700HRT-38699,POLYESTER FABRIC 280CM HRT-38699,HAR,1.80,3.75
1,1,BELKI,2021-06-01,F/193563/ZAS/CZA,280/HRT-38699,38700HRT-38699,POLYESTER FABRIC 280CM HRT-38699,HAR,1.80,3.75
2,1,BELKI,2021-06-01,193563/ZAS/J.RÓŻ,280/HRT-38707,38700HRT-38707,POLYESTER FABRIC 280CM HRT-38707,HAR,1.85,3.70
3,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,2.20,3.85
4,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,4.40,3.85
...,...,...,...,...,...,...,...,...,...,...
2183,1,BELKI,2022-05-27,7173/ŻORŻ/NATU,300/VBY004,7173 V110,VBY004 300CM WITH LB,EYL,2.15,3.20
2184,1,BELKI,2022-05-27,HAMILTON/DIM/111,320/BŁĘ+GRAN1 DIMOUT,HAMILTON 111,"320CM HAMILTON BLACKOUT 111, 247 GSM, DOUBLE SIDE",ADE,1.60,8.60
2185,1,BELKI,2022-05-27,VARNA/ZAS/POP+CZ,140 12Z LONETA,VARNA COL12ROJO,VARNA COL12 140CM ROJO LONETA,LLA,10.50,2.00
2187,1,BELKI,2022-05-30,1403/ZAS/LATTE,300/35-BEJ,ADA 35-BEJ,ADA FONLUK 195GSM COL 35-BEJ 300CM,KAR,3.00,2.40


In [98]:
df_slownik.loc[df_slownik['NAZWA'].str.contains('VELVET/280/J.K')]#.loc[df_slownik['RODZAJ']=='140X180 EUR']

,KOD_TEMA,NAZWA,RODZAJ,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,SYMBOL_IMP
110844,224071,VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300GSM 280CM COL.(ZF-00)ZF-68,"3,7",FRENCH ZF-68
117815,308282,F/VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300 GSM 280CM COL.ZF-00,"3,56",FRENCH VELV.280
148201,335827,F/VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300 GSM 280CM COL.ZF-00,"3,56",FRENCH VELV.280
191090,409891,F/VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300GSM 280CM COL.(ZF-00)ZF-68,"3,7",FRENCH ZF-68


In [201]:
#_.merge(df_test[['Z_DNIA','name_without_f','RODZAJ','ILOSC','NAZWA', 'ZNACZNIK']], how='outer', left_on=['Z_DNIA','NAZWA1','RODZAJ1', 'ZNACZNIK'], right_on=['Z_DNIA','name_without_f','RODZAJ', 'ZNACZNIK'])

In [ ]:
SZ6021/12059
SC6000057571

In [23]:
df_belki.loc[df_belki['FAKTURA']=='SC6000057571']

,FAKTURA,Z_DNIA,NAZWA1,RODZAJ1,CENA_Z1,ILOSC1,NAZWA2,RODZAJ2,CENA_Z2,ILOSC2,KOD_TEMA,VAT_P,CENA,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,SYMBOL_IMP,year,month,czy_z_f,name_tmp
25870,SC6000057571,2021-11-12,1403/ZAS/BRĄZ,300/27057,10.54,0.65,1403/ZAS/BRĄZ,300/27057,10.54,0.00,365941,23,22.90,1,KAR,ADA FONLUK 195GSM COL EVT-27057 300CM,2.85,ADA EVT-27057,2021,11,nie,1403/ZAS/BRĄZ
25871,SC6000057571,2021-11-12,1403/ZAS/BRĄZ,300/27057,10.54,0.00,F/1403/ZAS/BRĄZ,300/193 FIN,10.54,0.65,335479,23,22.90,1,,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,2.90,AT01403,2021,11,tak,1403/ZAS/BRĄZ
25888,SC6000057571,2021-11-12,1403/ZAS/LATTE,300/35-BEJ,10.54,2.25,1403/ZAS/LATTE,300/35-BEJ,10.54,0.00,364853,23,22.90,1,KAR,ADA FONLUK 195GSM COL 35-BEJ 300CM,2.85,ADA 35-BEJ,2021,11,nie,1403/ZAS/LATTE
25889,SC6000057571,2021-11-12,1403/ZAS/LATTE,300/35-BEJ,10.54,0.00,F/1403/ZAS/LATTE,300/35-BEJ,10.54,2.25,380700,23,22.90,1,KAR,ADA FONLUK 195GSM COL 35-BEJ 300CM,2.40,ADA 35-BEJ,2021,11,tak,1403/ZAS/LATTE
25890,SC6000057571,2021-11-12,1403/ZAS/LAWE,300/223,10.61,0.00,F/1403/ZAS/LAWE,300/223,10.61,1.10,399225,23,21.41,1,FIN,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,2.70,AT01403,2021,11,tak,1403/ZAS/LAWE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26227,SC6000057571,2021-11-12,VELVET/ZAS/J.K,145/ZF-01 783,8.98,0.00,F/VELVET/ZAS/J.K,145/ZF-00,8.98,1.75,334156,23,26.50,1,783,FRENCH VELVET 145 CM COL ZF-00,1.99,FRENCH ZF-00,2021,11,tak,VELVET/ZAS/J.K
26233,SC6000057571,2021-11-12,W015/70000/B,300/V100,6.71,3.70,W015/70000/B,300/V100,6.71,0.00,94355,23,13.60,60,EYL,300CM WITH LB,1.75,E015 V100,2021,11,nie,W015/70000/B
26234,SC6000057571,2021-11-12,W015/70000/B,300/V100,6.71,0.00,F/W015/70000/B,300/70000,6.71,3.70,85138,23,13.60,1,STS,,0.00,,2021,11,tak,W015/70000/B
26238,SC6000057571,2021-11-12,WALDO/ZAS/NATU,140/00 MOR,21.95,0.00,F/WALDO/ZAS/NAT,140/00,21.95,0.60,325247,23,44.80,1,MOR,FABRIC 140CM,5.90,3586.00,2021,11,tak,WALDO/ZAS/NAT


In [27]:
df_raport_belki2.loc[df_raport_belki2['NAZWA'].str.contains('1403/ZAS/BRĄZ')]

,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,SYMBOL_IMP,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
175,1,BELKI,2021-06-07,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,4.95,2.9
2972,1,BELKI,2021-09-15,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,2.35,2.9
3560,1,BELKI,2021-10-07,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,1.70,2.9
4689,1,BELKI,2021-11-10,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,7.80,2.9
4726,1,BELKI,2021-11-12,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,,0.65,2.9
5666,1,BELKI,2021-12-17,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,1.50,2.9
7626,1,BELKI,2022-02-23,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,1.60,2.9
8185,1,BELKI,2022-03-23,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,2.15,2.9


In [28]:
df_raport_belki2.loc[df_raport_belki2['Z_DNIA']==np.datetime64('2021-11-12')]

,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,SYMBOL_IMP,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
4724,1,BELKI,2021-11-12,1033/SREB,300/03 MOR,,,MOR,1.20,0.00
4725,1,BELKI,2021-11-12,1403/ZAS/BORD,300/27-BORDO,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,FIN,0.60,2.90
4726,1,BELKI,2021-11-12,1403/ZAS/BRĄZ,300/27057,AT01403,100%POLYESTER CURTAINING FABRIC WITHOUT LB COL...,,0.65,2.90
4727,1,BELKI,2021-11-12,1403/ZAS/C.MIĘT,300/29957,ADA EVT29957,ADA FONLUK BRUSHED EVT29957 195GSM 300CM,KAR,27.35,2.60
4728,1,BELKI,2021-11-12,1403/ZAS/C.PUDE,300/34-PUDRA,ADA 34-PUDRA,ADA FONLUK 195GSM COL 34-PUDRA 300CM,KAR,3.50,2.40
...,...,...,...,...,...,...,...,...,...,...
4824,1,BELKI,2021-11-12,VELVET/ZAS/J.K1,145/ZF-00,FRENCH ZF-00,FRENCH VELVET 145 CM COL ZF-00,783,1.75,1.99
4825,1,BELKI,2021-11-12,VILNIUS/100/K,140/1,VILNIUS 100 C1,COL 1 140 CM,ADE,0.70,13.50
4826,1,BELKI,2021-11-12,VILNIUS/110/C.TU,140/10,VILNIUS 110 C10,COL 10 140 CM,ADE,0.90,13.50
4827,1,BELKI,2021-11-12,W015/70000/B,300/70000,,,STS,3.70,0.00


In [78]:
list_of_index_to_add_belki=[]
for i in tqdm(df_raport_belki1.index):
    if df_raport_belki2.loc[(df_raport_belki2['Z_DNIA']==df_raport_belki1.loc[i, 'Z_DNIA']) & (df_raport_belki2['NAZWA']==df_raport_belki1.loc[i, 'NAZWA']) & (df_raport_belki2['RODZAJ']==df_raport_belki1.loc[i, 'RODZAJ']) &\
                           (df_raport_belki2['ILOSC']==df_raport_belki1.loc[i, 'ILOSC'])].empty:
        list_of_index_to_add_belki=np.append(list_of_index_to_add_belki, i)

100%|█████████████████████████████████████████████████████████████████████████████| 1636/1636 [00:06<00:00, 247.08it/s]


In [62]:
list_of_index_to_add

array([  35.,   36.,   37.,   38.,   68.,   69.,   70.,   71.,   72.,
         73.,   74.,  161.,  307.,  308.,  393.,  400.,  418.,  432.,
        436.,  437.,  438.,  439.,  440.,  441.,  479.,  480.,  515.,
        516.,  517.,  518.,  525.,  669.,  752.,  765.,  837.,  838.,
        839.,  843., 1013., 1122., 1146., 1147., 1173., 1246., 1328.,
       1348., 1354., 1396., 1397., 1425., 1427., 1443., 1473., 1560.])

In [82]:
list_of_index_to_add=[]
for i in tqdm(df_raport2.index):
    if df_raport1.loc[(df_raport1['Z_DNIA']==df_raport2.loc[i, 'Z_DNIA']) & (df_raport1['NAZWA']==df_raport2.loc[i, 'NAZWA']) & (df_raport1['RODZAJ']==df_raport2.loc[i, 'RODZAJ']) &\
                           (df_raport1['ILOSC']==df_raport2.loc[i, 'ILOSC'])].empty:
        list_of_index_to_add=np.append(list_of_index_to_add, i)

100%|█████████████████████████████████████████████████████████████████████████████| 8750/8750 [00:33<00:00, 263.24it/s]


In [79]:
len(df_raport_belki2)+len(list_of_index_to_add_belki)

10486

In [136]:
len(df_raport1)#+len(list_of_index_to_add)

8813

In [159]:
len(df_raport2)

8750

In [75]:
len(df_belki.loc[df_belki['ILOSC2']>0].loc[df_belki['ILOSC1']==0].loc[(df_belki['NAZWA1'].str[:2]=='F/')|(df_belki['NAZWA2'].str[:2]=='F/')])

10628

In [76]:
len(df.loc[df['ILOSC2']>0].loc[df['ILOSC1']==0].loc[(df['NAZWA1'].str[:2]=='F/')|(df['NAZWA2'].str[:2]=='F/')])

8832

In [132]:
df.loc[df['ILOSC2']>0].loc[df['ILOSC1']==0].loc[(df['NAZWA1'].str[:2]=='F/')|(df['NAZWA2'].str[:2]=='F/')]['ILOSC2'].sum()

19033.91

In [91]:
df_raport1['ILOSC'].sum()

19009.91

In [133]:
df_raport1.append(df_raport2.loc[list_of_index_to_add])['ILOSC'].sum()

19152.41

In [123]:
df_belki.loc[df_belki['ILOSC2']>0].loc[df_belki['ILOSC1']==0].loc[(df_belki['NAZWA1'].str[:2]=='F/')|(df_belki['NAZWA2'].str[:2]=='F/')]['ILOSC2'].sum()

31923.5

In [95]:
df_raport_belki2['ILOSC'].sum()

30981.5

In [99]:
df_raport_belki2.append(df_raport_belki1.loc[list_of_index_to_add_belki])['ILOSC'].sum()

31241.95

In [107]:
tmp1=df_belki.loc[df_belki['ILOSC2']>0].loc[df_belki['ILOSC1']==0].loc[(df_belki['NAZWA1'].str[:2]=='F/')|(df_belki['NAZWA2'].str[:2]=='F/')][['Z_DNIA','NAZWA_IMP','ILOSC2']].reset_index()

In [106]:
tmp2=df_raport_belki2.append(df_raport_belki1.loc[list_of_index_to_add_belki])[['Z_DNIA','NAZWA_IMP','ILOSC']]
tmp2['aa']='-'

In [109]:
tmp3=tmp1.merge(tmp2, how='outer', left_on=['Z_DNIA','NAZWA_IMP','ILOSC2'], right_on=['Z_DNIA','NAZWA_IMP','ILOSC'])

In [111]:
tmp3.loc[tmp3['aa'].isna()]

,index,Z_DNIA,NAZWA_IMP,ILOSC2,ILOSC,aa
680,2973,2021-06-18,FRENCH VELVET 300GSM 145CM COL EU-5C,1.55,NaN,NaN
1171,5321,2021-07-07,5331 col 10 300cm WITH LB,4.35,NaN,NaN
1851,9033,2021-08-03,ADA FONLUK BRUSHED 195GSM EVT25589 300CM,0.60,NaN,NaN
2103,9926,2021-08-09,POLYESTER FABRIC 280CM HRT-38709,3.00,NaN,NaN
2689,12787,2021-08-26,320CM WITH LB,5.60,NaN,NaN
2895,13703,2021-09-01,ADA FONLUK BRUSHED 195GSM EVT25589 300CM,2.00,NaN,NaN
3153,14952,2021-09-09,FRENCH VELVET 300GSM 145CM COL EU-02A,3.00,NaN,NaN
3457,16539,2021-09-20,51061 col 101 300cm,0.60,NaN,NaN
3671,17610,2021-09-27,ADA FONLUK 33-SIYAH 195GSM 300CM,0.50,NaN,NaN
4679,23340,2021-10-28,FRENCH VELVET 300GSM 280CM COL ZF-18,5.30,NaN,NaN


In [138]:
df_belki.loc[df_belki['Z_DNIA']==np.datetime64('2022-05-13')].loc[df_belki['NAZWA_IMP']=='TD12407LOZ2800043 SELVA LONETA 14TAP 140CM']

,FAKTURA,Z_DNIA,NAZWA1,RODZAJ1,CENA_Z1,ILOSC1,NAZWA2,RODZAJ2,CENA_Z2,ILOSC2,KOD_TEMA,VAT_P,CENA,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,SYMBOL_IMP,year,month,czy_z_f,name_tmp
55195,SC6000065246,2022-05-13,F/SELVA/Z/BEŻ+ZI,140 C43 PICAS LLA,13.48,0.0,SELVA/ZAS/BEŻ+ZI,140 C43 LONETA,13.48,2.9,313645,23,31.6,1,LLA,TD12407LOZ2800043 SELVA LONETA 14TAP 140CM,2.5,SELVA LONETA,2022,5,tak,SELVA/Z/BEŻ+ZI
55198,SC6000065246,2022-05-13,F/SELVA/Z/BEŻ+ZI,140 C43 PICAS LLA,13.48,0.0,SELVA/ZAS/BEŻ+ZI,140 C43 LONETA,13.48,2.9,313645,23,31.6,1,LLA,TD12407LOZ2800043 SELVA LONETA 14TAP 140CM,2.5,SELVA LONETA,2022,5,tak,SELVA/Z/BEŻ+ZI
55233,SC6000065247,2022-05-13,SELVA/ZAS/BEŻ+ZI,140 C43 LONETA,13.48,2.9,SELVA/ZAS/BEŻ+ZI,140 C43 LONETA,13.48,0.0,313645,23,31.6,1,LLA,TD12407LOZ2800043 SELVA LONETA 14TAP 140CM,2.5,SELVA LONETA,2022,5,nie,SELVA/ZAS/BEŻ+ZI
55236,SC6000065247,2022-05-13,SELVA/ZAS/BEŻ+ZI,140 C43 LONETA,13.48,2.9,SELVA/ZAS/BEŻ+ZI,140 C43 LONETA,13.48,0.0,313645,23,31.6,1,LLA,TD12407LOZ2800043 SELVA LONETA 14TAP 140CM,2.5,SELVA LONETA,2022,5,nie,SELVA/ZAS/BEŻ+ZI


In [140]:
df_raport_belki2.loc[df_raport_belki2['Z_DNIA']>=np.datetime64('2022-05-01')].loc[df_raport_belki2['Z_DNIA']<=np.datetime64('2022-05-15')]

,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,SYMBOL_IMP,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
9619,60,Hit BAZA,2022-05-04,10003/B,280/ V05 WHITE,ARM10003 V99V05,"290CM, L/B, YARN V99, BASE V05 WHITE",NET,3.45,3.10
9620,1,BELKI,2022-05-04,10096/ZAS/SREB,300/V012,10096 V-012,300CM,EYL,1.75,3.10
9621,1,BELKI,2022-05-04,10630/ETA/NAT+SR,300/999,J10630 999,WITH LUREX PLAIN WITH LB 300CM,URK,0.85,6.70
9622,1,BELKI,2022-05-04,1315/ETA/NATU,300/V-101,AT01315 V-101,300CM 100% PLY CURTAIN FABRIC WITH LB,FIN,2.05,2.80
9623,1,BELKI,2022-05-04,1403/ZAS/CAPU,300/21068,ADA EVT21068,ADA FONLUK 195GSM COL EVT21068 300CM,KAR,0.60,2.65
...,...,...,...,...,...,...,...,...,...,...
9722,1,BELKI,2022-05-13,VELVET/280/BEŻ,280/ZF-25,FRENCH VELV.280,FRENCH VELVET 300 GSM 280CM COL.ZF-25,783,3.20,3.56
9723,1,BELKI,2022-05-13,VELVET/280/KAKA1,280/ZF-06,FRENCH ZF-04,FRENCH VELVET 300GSM 280CM COL ZF-04,783,4.60,3.56
9724,1,BELKI,2022-05-13,VELVET/280/STAL,280/ZF-27,FRENCH VELV.280,FRENCH VELVET 300 GSM 280CM COL.ZF-27,783,2.30,3.56
9725,1,BELKI,2022-05-13,VELVET/ZAS/GRAN,145/EU-30-1,FRENCH EU-30-1C,FRENCH VELVET 145 CM COL EU-30-1C,783,28.00,1.99


In [153]:
df_raport_belki2.loc[df_raport_belki2['Z_DNIA']==np.datetime64('2022-05-13')]['ILOSC'].sum()

266.09999999999997

In [154]:
tmp1.loc[tmp1['Z_DNIA']==np.datetime64('2022-05-13')]['ILOSC2'].sum()

271.9

In [155]:
df_belki.loc[df_belki['ILOSC2']>0].loc[df_belki['ILOSC1']==0].loc[(df_belki['NAZWA2'].str[:2]=='F/')].loc[df_belki['Z_DNIA']==np.datetime64('2022-05-13')]['ILOSC2'].sum()

266.09999999999997

In [158]:
df_belki.loc[df_belki['ILOSC2']>0].loc[df_belki['ILOSC1']==0].loc[(df_belki['NAZWA2'].str[:2]=='F/')].loc[df_belki['Z_DNIA']==np.datetime64('2022-05-13')].sort_values(by='ILOSC2')#['NAZWA2'].unique()

,FAKTURA,Z_DNIA,NAZWA1,RODZAJ1,CENA_Z1,ILOSC1,NAZWA2,RODZAJ2,CENA_Z2,ILOSC2,KOD_TEMA,VAT_P,CENA,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,SYMBOL_IMP,year,month,czy_z_f,name_tmp
55258,SC6000065244,2022-05-13,SIBEL/ZAS/C.ZIEL,300 VELVET 260 GSM,13.64,0.0,F/SIBEL/ZAS/C.ZI,300 VELVET 260 GSM,13.64,2.1,394293,23,29.90,1,906,300CM 260GSM DARK GREEN,2.60,260GSM VELVET,2022,5,nie,SIBEL/ZAS/C.ZIEL
55312,SC6000065244,2022-05-13,VELVET/280/STAL,280/ZF-27,19.42,0.0,F/VELVET/280/STA,280/ZF-27,19.42,2.3,313136,23,47.00,1,783,FRENCH VELVET 300 GSM 280CM COL.ZF-27,3.56,FRENCH VELV.280,2022,5,nie,VELVET/280/STAL
55174,SC6000065209,2022-05-13,DARK/ZAS/BEŻ,300/66 DIMOUT,21.91,0.0,F/DARK/ZAS/BEŻ,300/66 DIMOUT,21.91,2.5,392038,23,45.50,1,ELN,300CM DARK BLACKOUT 220GSM,4.80,DARK BLACKO/V66,2022,5,nie,DARK/ZAS/BEŻ
55199,SC6000065245,2022-05-13,F/SELVA/Z/BEŻ+ZI,140 C43 PICAS LLA,13.48,0.0,F/SELVA/Z/BEŻ+ZI,140 C43 PICAS LLA,13.48,2.9,336079,23,0.00,1,LLA,TJ12407PKZ1400043 SELVA PICAS 14TAP 140CM,3.10,SELVA PICAS,2022,5,tak,SELVA/Z/BEŻ+ZI
55194,SC6000065245,2022-05-13,F/SELVA/Z/BEŻ+ZI,140 C43 PICAS LLA,13.48,0.0,F/SELVA/Z/BEŻ+ZI,140 C43 PICAS LLA,13.48,2.9,336079,23,0.00,1,LLA,TJ12407PKZ1400043 SELVA PICAS 14TAP 140CM,3.10,SELVA PICAS,2022,5,tak,SELVA/Z/BEŻ+ZI
55271,SC6000065244,2022-05-13,VELVET/280/BEŻ,280/ZF-25,14.77,0.0,F/VELVET/280/BEŻ,280/ZF-25,14.77,3.2,310892,23,31.90,1,783,FRENCH VELVET 300 GSM 280CM COL.ZF-25,3.56,FRENCH VELV.280,2022,5,nie,VELVET/280/BEŻ
55283,SC6000065244,2022-05-13,VELVET/280/KAKA1,280/ZF-06,19.42,0.0,F/VELVET/28/KAK1,280/ZF-04,19.42,4.6,403240,23,47.00,1,783,FRENCH VELVET 300GSM 280CM COL ZF-04,3.56,FRENCH ZF-04,2022,5,nie,VELVET/280/KAKA1
55344,SC6000065244,2022-05-13,VELVET/ZAS/KAK2,145/ZF-07,12.06,0.0,F/VELVET/ZA/KAK2,145/ZF-07,12.06,4.9,349955,23,27.50,1,783,FRENCH VELVET 145 CM COL ZF-07,1.99,FRENCH ZF-07,2022,5,nie,VELVET/ZAS/KAK2
55212,SC6000065244,2022-05-13,PERU/ORG/B,300/9071 V100,6.02,0.0,F/PERU/ORG/B,300/WHITE LAR,6.02,7.6,333490,23,13.50,1,,FANCY FABRIC 300CM WITH LB,1.40,PIRILTI WHITE,2022,5,nie,PERU/ORG/B
55079,SC6000065244,2022-05-13,150547/ZAS/POPI,310/109 EVI,0.44,0.0,F/150547/ZAS/POP,310/109,0.44,8.5,373533,23,101.73,1,EVI,J150547 COL 310CM WITH LB,11.40,J150547 109,2022,5,nie,150547/ZAS/POPI
